# Simulate Ozone effet on differential color light curves

- author Sylvie Dagoret-Campagne
- creation date : 2024/08/31
- last update : 2024/08/31 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'


In [ ]:
from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

In [ ]:
machine_name = os.uname().nodename
path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

## Imports dedicated to this work

- import the atmospheric transparency emulator (instead of using libradtran code).
- import rubin sim
- import libPhotometricCorrections : encapsulate uninteresting calculation details

### libradtran Emulator

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("AUXTEL")
#emul = ObsAtmo("LSST")

In [ ]:
WL = emul.GetWL()

#### Library to fit atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

#### Library that encapsulate calculations for Photometric correction

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections_auxtel import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

#### library rubin_sim defining LSST parameters, namely for photometric calculations

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

## Configuration

In [ ]:
am0 =1.20    # airmass
pwv0 = 3.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.04 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

### Initialisation of Atmospheric corrections

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### Check standard atmosphere

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
axs.plot(pc.WL,pc.atm_std,'k-')
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Standard atmosphere transmission")

### Check LSST instrument throughput

Photometric Correction package should find the instrumental passband of LSST

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Instrument throughput (auxtel)")

### Check LSST standard Filter throughputs

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total filter throughput (auxtel)")

## Variation of PWV

In [ ]:
am = 1.2
oz = 300
tau= 0.04
beta = 1
all_pwv = np.arange(0,10,0.5)
NPWV = len(all_pwv)

In [ ]:
pc.CalculateMultiObs(am,all_pwv,oz,tau,beta)

### PWV variation :  Observed filter and normalized response

In [ ]:
NOBS = len(all_pwv)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(12,4))

# Figure 1
axs=fig.add_subplot(1,2,1)
for index,pwv in enumerate(all_pwv):
  
    atm = pc.coll_atm_nonstd[index]
    
    label = f"pwv={pwv:.1f}" 
    axs.plot(pc.WL,atm,color=all_colors[index],label=label,lw=0.5)
   
axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("standard and observed transmission")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])



# Figure 2
axs=fig.add_subplot(1,2,2)

all_linestyles = ['-','--','-.',':','-','--','-.',':','-','--','-.',':','-','--','-.',':']

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


for idx_pwv,pwv in enumerate(all_pwv):
    
    label = f"pwv={pwv:.1f}"
    
    for ifilt,f in enumerate(filter_tagnames):
        
        the_x=pc.WL
        the_y=pc.coll_phiArray_nonstd[idx_pwv][ifilt,:]
       
        
        if ifilt==1:
            axs.plot(the_x,the_y,color=all_colors[idx_pwv],linestyle="-",label=label )
        else:
            axs.plot(the_x,the_y,color=all_colors[idx_pwv],linestyle="-")

axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("Normalized observed transmission")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  


plt.tight_layout()
plt.show()



## SED

In [ ]:
# Find the throughputs directory 
#fdir = os.getenv('RUBIN_SIM_DATA_DIR')
fdir = get_data_dir()
if fdir is None:  #environment variable not set
    fdir = os.path.join(os.getenv('HOME'), 'rubin_sim_data')

In [ ]:
seddir = os.path.join(fdir, 'pysynphot', 'pickles')

In [ ]:
seddir_uvi = os.path.join(seddir,"dat_uvi")
seddir_uvk = os.path.join(seddir,"dat_uvk")

In [ ]:
all_pickles_uvi = sorted(os.listdir(seddir_uvi))
all_pickles_uvk = sorted(os.listdir(seddir_uvk))

In [ ]:
file_ref = os.path.join(seddir_uvk, "pickles_uk.fits")

In [ ]:
hdul = fits.open(file_ref)

In [ ]:
df_pickle = pd.DataFrame(hdul[1].data)

In [ ]:
df_pickle

In [ ]:
NSED = len(df_pickle)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 0.6), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=NSED)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='spectral type')

# change the number of labels
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]
cbar.ax.locator_params(axis='x', nbins=4*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]

# rename the labels
Nlabels = len(labels)
for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_pickle.loc[the_label,"SPTYPE"]
cbar.ax.set_xticklabels(labels,rotation=45);
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
for index in np.arange(NSED):
    filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
    fullfilename = os.path.join(seddir_uvk,filename) 
    hdul = fits.open(fullfilename)
    dff = pd.DataFrame(hdul[1].data)
    if index <= 103 or index >= 105:
        ax.plot(dff.WAVELENGTH,dff.FLUX,color=all_colors[index])
ax.set_yscale('log')
ax.set_xlim(3000.,11000)
ax.set_ylim(1e-11,1e-6)
ax.grid()
ax.set_title("Pickles $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (\AA$)")
#plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=0.8)
#ax.collections[-1].colorbar


## Convert SED in rubinsim format

In [ ]:
all_sed = []
rmag = 20.0
for index in np.arange(NSED):
    spectype = df_pickle.loc[index,"SPTYPE"].strip()
    filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
    fullfilename = os.path.join(seddir_uvk,filename) 
    hdul = fits.open(fullfilename)
    dff = pd.DataFrame(hdul[1].data)
    wl= dff.WAVELENGTH.values
    flux = dff.FLUX.values
    #wavelen (nm)
    #flambda (ergs/cm^2/s/nm)
    the_sed = Sed(wavelen=wl/10., flambda=flux*10., name=spectype)
    flux_norm = the_sed.calc_flux_norm(rmag, pc.bandpass_total_std['r'])
    the_sed.multiply_flux_norm(flux_norm)
    all_sed.append(the_sed) 

### Check the calculation of the Magnitude and the ADC counts (observation)

In [ ]:
the_sed1 = all_sed[49]
flux_norm = the_sed1.calc_flux_norm(rmag, pc.bandpass_total_std['r'])
the_sed1.multiply_flux_norm(flux_norm)

the_sed2 = all_sed[129]
flux_norm = the_sed2.calc_flux_norm(rmag, pc.bandpass_total_std['r'])
the_sed2.multiply_flux_norm(flux_norm)

In [ ]:
fig,(ax,ax2) = plt.subplots(1,2,figsize=(16,6))
ax.plot(the_sed1 .wavelen,the_sed1 .flambda,"b-",label=the_sed1.name)
ax.plot(the_sed2.wavelen,the_sed2.flambda,"r-",label=the_sed2.name)
ax.set_yscale("log")
ax.legend()
ax.set_ylim(1e-17,1e-14)
ax.set_xlim(300.,2000.)
ax.set_title("Pickles $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (\AA$)")


ax2.plot(the_sed1 .wavelen,the_sed1.fnu,"b-",label=the_sed1.name)
ax2.plot(the_sed2.wavelen,the_sed2.fnu,"r-",label=the_sed2.name)
ax2.set_yscale("log")
ax2.legend()
ax2.set_ylim(1e-7,1e-3)
ax2.set_xlim(300.,2000.)
ax2.set_title("Pickles $F_\\nu$")
ax2.set_ylabel("$F_\\nu$")
ax2.set_xlabel("$\lambda \, (\AA$)")

ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])
    
ax4 = ax2.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax4.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax4.set_yticks([])

In [ ]:
mags_std1 = {}
for index,f in enumerate(filter_tagnames) :
    mags_std1[f] = the_sed1.calc_mag(pc.bandpass_total_std[f])
mags_std1["g-r"] = mags_std1["g"]- mags_std1["r"]  
mags_std1["z-y"] = mags_std1["z"]- mags_std1["y"]   
df1 = pd.DataFrame(mags_std1, index=[the_sed1.name]).T

mags_std2 = {}
for index,f in enumerate(filter_tagnames) :
    mags_std2[f] = the_sed2.calc_mag(pc.bandpass_total_std[f])
mags_std2["g-r"] = mags_std2["g"]- mags_std2["r"]  
mags_std2["z-y"] = mags_std2["z"]- mags_std2["y"]   
df2 = pd.DataFrame(mags_std2, index=[the_sed2.name]).T
dfa =pd.concat([df1,df2],axis=1)
dfa

In [ ]:
photoparams = set_photometric_parameters(30, 1 , readnoise=None)
adcs_stdmag1 = {}
for index,f in enumerate(filter_tagnames) :
    adcs_stdmag1[f] = -2.5*np.log10(the_sed1.calc_adu(pc.bandpass_total_std[f],photoparams))
adcs_stdmag1["g-r"] = adcs_stdmag1["g"]- adcs_stdmag1["r"]  
adcs_stdmag1["z-y"] = adcs_stdmag1["z"]- adcs_stdmag1["y"]      
df1 = pd.DataFrame(adcs_stdmag1, index=[the_sed1.name]).T

adcs_stdmag2 = {}
for index,f in enumerate(filter_tagnames) :
    adcs_stdmag2[f] = -2.5*np.log10(the_sed2.calc_adu(pc.bandpass_total_std[f],photoparams))
adcs_stdmag2["g-r"] = adcs_stdmag2["g"]- adcs_stdmag2["r"]  
adcs_stdmag2["z-y"] = adcs_stdmag2["z"]- adcs_stdmag2["y"]      
df2 = pd.DataFrame(adcs_stdmag2, index=[the_sed2.name]).T

dfb =pd.concat([df1,df2],axis=1)
dfb

## Simulate multi atmospheres

In [ ]:
am = 1.2
all_oz = np.arange(0,610.,10.)
tau= 0.04
beta = 1
pwv = 3.0

NOZ = len(all_oz)

In [ ]:
pc.CalculateMultiObs(am,pwv,all_oz,tau,beta)

In [ ]:
NOBS = len(all_oz)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(12,8))

# Figure 1
axs=fig.add_subplot(1,1,1)
for index,oz in enumerate(all_oz):

    #atm = pc.coll_bandpass_total_nonstd[index]
    atm = pc.coll_atm_nonstd[index]
    
    label = f"pwv={pwv:.1f}" 
    axs.plot(pc.WL,atm,color=all_colors[index],label=label,lw=0.5)
   
axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("standard and observed transmission")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()


## Note the passbands for each of the OZ conditions are in pc.coll_bandpass_total_nonstd

In [ ]:
len(pc.coll_bandpass_total_nonstd)

In [ ]:
pc.coll_bandpass_total_nonstd[0]["u"].wavelen[:5]

In [ ]:
pc.coll_bandpass_total_nonstd[0]["u"].sb[:9]

In [ ]:
NOBS = len(all_oz)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(12,8))

# Figure 1
axs=fig.add_subplot(1,1,1)
for index,oz in enumerate(all_oz):
  
    atm_bands = pc.coll_bandpass_total_nonstd[index]
    
    label = f"pwv={pwv:.1f}" 

    for f in filter_tagnames: 
        axs.plot(atm_bands[f].wavelen,atm_bands[f].sb,color=all_colors[index],label=label,lw=0.5)
   
axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("standard and observed transmission")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()


## Compute the Magnitudes in all these transmission

- in all_df_magcol

In [ ]:
all_df_magcol = []
for index,oz in enumerate(all_oz):
    atm_bands = pc.coll_bandpass_total_nonstd[index]
    mags_oz1 = {}
    mags_oz2 = {}
    madus_oz1 = {}
    madus_oz2 = {}
    for f in filter_tagnames: 
        mags_oz1[f] = the_sed1.calc_mag(atm_bands[f])
        mags_oz2[f] = the_sed2.calc_mag(atm_bands[f])
        madus_oz1[f] = -2.5*np.log10(the_sed1.calc_adu(atm_bands[f],photoparams))
        madus_oz2[f] = -2.5*np.log10(the_sed2.calc_adu(atm_bands[f],photoparams))
    # AB colors for star 1
    mags_oz1["g-r"] = mags_oz1["g"]- mags_oz1["r"]  
    mags_oz1["z-y"] = mags_oz1["z"]- mags_oz1["y"] 
    # AB colors for star 2
    mags_oz2["g-r"] = mags_oz2["g"]- mags_oz2["r"]  
    mags_oz2["z-y"] = mags_oz2["z"]- mags_oz2["y"] 
    # ADU mag colors for star 1
    # MADU colors for star 1
    madus_oz1["g-r"] = madus_oz1["g"]- madus_oz1["r"]  
    madus_oz1["z-y"] = madus_oz1["z"]- madus_oz1["y"] 
    # MADU colors for star 2
    madus_oz2["g-r"] = madus_oz2["g"]- madus_oz2["r"]  
    madus_oz2["z-y"] = madus_oz2["z"]- madus_oz2["y"] 
        
    df1 = pd.DataFrame(mags_oz1, index=["abmag_1"]).T
    df2 = pd.DataFrame(mags_oz2, index=["abmag_2"]).T
    df3 = pd.DataFrame(madus_oz1, index=["instmag_1"]).T
    df4 = pd.DataFrame(madus_oz2, index=["instmag_2"]).T
    df = pd.concat([df1,df2,df3,df4],axis=1)
    all_df_magcol.append(df)
   

In [ ]:
all_df_magcol[0]

## Compute the relative colors of the pair

In [ ]:
df_diffcol = pd.DataFrame(columns=["oz","Cgr1_Cgr2_abmag","Cgr1_Cgr2_instmag"])
for index,oz in enumerate(all_oz):
    the_df = all_df_magcol[index]
    DCgrabmag = the_df.loc["g-r","abmag_1"] - the_df.loc["g-r","abmag_2"]
    DCgradumag = the_df.loc["g-r","instmag_1"] - the_df.loc["g-r","instmag_2"]
    df_diffcol.loc[index] = [oz,DCgrabmag,DCgradumag]
    
df_diffcol["Cgr1_Cgr2_abmag_rel"] =  (df_diffcol["Cgr1_Cgr2_abmag"] - df_diffcol.loc[0,"Cgr1_Cgr2_abmag"])*1000
df_diffcol["Cgr1_Cgr2_instmag_rel"] =  (df_diffcol["Cgr1_Cgr2_instmag"] - df_diffcol.loc[0,"Cgr1_Cgr2_instmag"])*1000
# differentiation
dx = np.diff(df_diffcol.oz.values).mean()
dy1 = np.diff(df_diffcol["Cgr1_Cgr2_abmag_rel"].values)
dy2 = np.diff(df_diffcol["Cgr1_Cgr2_instmag_rel"].values)
g_r0 = df_diffcol.loc[0,"Cgr1_Cgr2_abmag"]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
df_diffcol.plot(x="oz",y="Cgr1_Cgr2_abmag_rel",ax=ax,color="r",label="AB mag color diff")
df_diffcol.plot(x="oz",y="Cgr1_Cgr2_instmag_rel",ax=ax,color="b",label="Inst mag color diff")

ax.set_xlabel("Ozone (DU)")
ax.set_ylabel("$C_{gr1}-C_{gr2}$ (mmag)",color="b")
ax.set_title(f"Relative differential color bias")
#ax.yaxis.label.set_color('r')
ax.tick_params(axis='y', colors='b')

textstr = '\n'.join((" relative $C_{gr1}-C_{gr2} = $"+ f"{g_r0:.2f} mag","for Ozone = 0 DU")) 
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='white', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0.1, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)

ax2 = ax.twinx()
ax2.plot(df_diffcol.oz.values[:-1],dy1/dx,":",color="grey",label="dC/doz(ABmag)")
ax2.plot(df_diffcol.oz.values[:-1],dy2/dx,":",color="grey",label="dC/doz(Instmag)")
ax2.set_ylabel("$d(\Delta C_{gr})/d(oz)$ (mmag)",color="grey")
ax2.set_ylim(-.026,-0.024)
ax2.tick_params(axis='y', colors='grey')

In [ ]:
dx = np.diff(df_diffcol.oz.values).mean()

In [ ]:
dy1 = np.diff(df_diffcol["Cgr1_Cgr2_abmag_rel"].values)
dy2 = np.diff(df_diffcol["Cgr1_Cgr2_instmag_rel"].values)

In [ ]:
len(dy1/dx)

In [ ]:
dy2/dx